In [1]:
install.packages("tidyverse")

Installing package into ‘/md01/Huanga/R/x86_64-redhat-linux-gnu-library/3.6’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rvest’, ‘xml2’




In [9]:
install.packages("BiocManager")

Installing package into ‘/md01/Huanga/R/x86_64-redhat-linux-gnu-library/3.6’
(as ‘lib’ is unspecified)



In [2]:
library(tidyverse)

── Attaching packages ────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.1     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ───────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

